## BLERSSI  Seldon serving

## Clone Cisco Kubeflow Starter pack repository

In [1]:
BRANCH_NAME="dev" #Provide git branch name "master" or "dev"
! git clone -b $BRANCH_NAME https://github.com/CiscoAI/cisco-kubeflow-starter-pack.git

Cloning into 'cisco-kubeflow-starter-pack'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 4630 (delta 16), reused 44 (delta 11), pack-reused 4567
Receiving objects: 100% (4630/4630), 17.61 MiB | 48.72 MiB/s, done.
Resolving deltas: 100% (1745/1745), done.


## Install the required packages

In [2]:
! pip install pandas sklearn seldon_core dill alibi==0.3.2 --user

     |████████████████████████████████| 10.1 MB 21.3 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 134.7 MB/s eta 0:00:01
     |████████████████████████████████| 177 kB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 103 kB/s  eta 0:00:01
     |████████████████████████████████| 6.8 MB 135.5 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 16.7 MB/s  eta 0:00:01
     |████████████████████████████████| 75 kB 3.9 MB/s s eta 0:00:01
     |████████████████████████████████| 88 kB 21.2 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 24.3 MB/s  eta 0:00:01
     |████████████████████████████████| 72 kB 2.4 MB/s s eta 0:00:01
     |████████████████████████████████| 80 kB 31.8 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 27.1 MB/s  eta 0:00:01
     |████████████████████████████████| 115 kB 158.3 MB/s eta 0:00:01
     |████████████████████████████████| 12.4 MB 71.0 MB/s eta 0:00:01
     |██████████████████████████

## Restart Notebook kernel

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

## Import Libraries

In [1]:
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import pandas as pd
import numpy as np
import shutil
import yaml
import random
import re
import os
import dill
import logging
import requests
import json
from time import sleep
from sklearn.preprocessing import OneHotEncoder
from alibi.explainers import AnchorTabular

from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException

k8s_config.load_incluster_config()
api_client = k8s_client.CoreV1Api()
custom_api=k8s_client.CustomObjectsApi()

## Get Namespace
Get current k8s namespace

In [2]:
def is_running_in_k8s():
    return os.path.isdir('/var/run/secrets/kubernetes.io/')

def get_current_k8s_namespace():
    with open('/var/run/secrets/kubernetes.io/serviceaccount/namespace', 'r') as f:
        return f.readline()

def get_default_target_namespace():
    if not is_running_in_k8s():
        return 'default'
    return get_current_k8s_namespace()

namespace = get_default_target_namespace()
print(namespace)

anonymous


## Check GPUs availability

In [3]:
gpus = len(tf.config.experimental.list_physical_devices('GPU'))
if gpus == 0:
    print("Model will be trained using CPU")
elif gpus >= 0:
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    tf.config.experimental.list_physical_devices('GPU')
    print("Model will be trained using GPU")

Model will be trained using CPU


## Declare Variables

In [4]:
path="cisco-kubeflow-starter-pack/apps/networking/ble-localization/onprem"
BLE_RSSI = pd.read_csv(os.path.join(path, "data/iBeacon_RSSI_Labeled.csv")) #Labeled dataset

# Configure model options
TF_DATA_DIR = os.getenv("TF_DATA_DIR", "/tmp/data/")
TF_MODEL_DIR = os.getenv("TF_MODEL_DIR", "blerssi/")
TF_EXPORT_DIR = os.getenv("TF_EXPORT_DIR", "blerssi/")
TF_MODEL_TYPE = os.getenv("TF_MODEL_TYPE", "DNN")
TF_TRAIN_STEPS = int(os.getenv("TF_TRAIN_STEPS", 5000))
TF_BATCH_SIZE = int(os.getenv("TF_BATCH_SIZE", 128))
TF_LEARNING_RATE = float(os.getenv("TF_LEARNING_RATE", 0.001))


# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
def make_feature_cols():
  input_columns = [tf.feature_column.numeric_column(k) for k in FEATURES]
  return input_columns

## BLERSSI Input Dataset
### Attribute Information
location: The location of receiving RSSIs from ibeacons b3001 to b3013; 
          symbolic values showing the column and row of the location on the map (e.g., A01 stands for column A, row 1).
date: Datetime in the format of ‘d-m-yyyy hh:mm:ss’
b3001 - b3013: RSSI readings corresponding to the iBeacons; numeric, integers only.

In [5]:
BLE_RSSI.head(10)

,location,date,b3001,b3002,b3003,b3004,b3005,b3006,b3007,b3008,b3009,b3010,b3011,b3012,b3013
0,O02,10-18-2016 11:15:21,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
1,P01,10-18-2016 11:15:19,-200,-200,-200,-200,-200,-78,-200,-200,-200,-200,-200,-200,-200
2,P01,10-18-2016 11:15:17,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
3,P01,10-18-2016 11:15:15,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
4,P01,10-18-2016 11:15:13,-200,-200,-200,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
5,P01,10-18-2016 11:15:11,-200,-200,-82,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
6,P01,10-18-2016 11:15:09,-200,-200,-80,-200,-200,-77,-200,-200,-200,-200,-200,-200,-200
7,P02,10-18-2016 11:15:07,-200,-200,-86,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200
8,R01,10-18-2016 11:15:05,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200
9,R01,10-18-2016 11:15:03,-200,-200,-200,-75,-200,-200,-200,-200,-200,-200,-200,-200,-200


## Definition of Serving Input Receiver Function

In [6]:
feature_columns =  make_feature_cols()
inputs = {}
for feat in feature_columns:
  inputs[feat.name] = tf.placeholder(shape=[None], dtype=feat.dtype)
serving_input_receiver_fn = tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)

## Train and Save BLE RSSI Model

In [7]:
# Feature columns
COLUMNS = list(BLE_RSSI.columns)
FEATURES = COLUMNS[2:]
LABEL = [COLUMNS[0]]

b3001 = tf.feature_column.numeric_column(key='b3001',dtype=tf.float64)
b3002 = tf.feature_column.numeric_column(key='b3002',dtype=tf.float64)
b3003 = tf.feature_column.numeric_column(key='b3003',dtype=tf.float64)
b3004 = tf.feature_column.numeric_column(key='b3004',dtype=tf.float64)
b3005 = tf.feature_column.numeric_column(key='b3005',dtype=tf.float64)
b3006 = tf.feature_column.numeric_column(key='b3006',dtype=tf.float64)
b3007 = tf.feature_column.numeric_column(key='b3007',dtype=tf.float64)
b3008 = tf.feature_column.numeric_column(key='b3008',dtype=tf.float64)
b3009 = tf.feature_column.numeric_column(key='b3009',dtype=tf.float64)
b3010 = tf.feature_column.numeric_column(key='b3010',dtype=tf.float64)
b3011 = tf.feature_column.numeric_column(key='b3011',dtype=tf.float64)
b3012 = tf.feature_column.numeric_column(key='b3012',dtype=tf.float64)
b3013 = tf.feature_column.numeric_column(key='b3013',dtype=tf.float64)
feature_columns = [b3001, b3002, b3003, b3004, b3005, b3006, b3007, b3008, b3009, b3010, b3011, b3012, b3013]

df_full = pd.read_csv(os.path.join(path, "data/iBeacon_RSSI_Labeled.csv")) #Labeled dataset

# Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full[FEATURES] = (df_full[FEATURES])/(-200)


#Output Data Preprocessing
dict = {'O02': 0,'P01': 1,'P02': 2,'R01': 3,'R02': 4,'S01': 5,'S02': 6,'T01': 7,'U02': 8,'U01': 9,'J03': 10,'K03': 11,'L03': 12,'M03': 13,'N03': 14,'O03': 15,'P03': 16,'Q03': 17,'R03': 18,'S03': 19,'T03': 20,'U03': 21,'U04': 22,'T04': 23,'S04': 24,'R04': 25,'Q04': 26,'P04': 27,'O04': 28,'N04': 29,'M04': 30,'L04': 31,'K04': 32,'J04': 33,'I04': 34,'I05': 35,'J05': 36,'K05': 37,'L05': 38,'M05': 39,'N05': 40,'O05': 41,'P05': 42,'Q05': 43,'R05': 44,'S05': 45,'T05': 46,'U05': 47,'S06': 48,'R06': 49,'Q06': 50,'P06': 51,'O06': 52,'N06': 53,'M06': 54,'L06': 55,'K06': 56,'J06': 57,'I06': 58,'F08': 59,'J02': 60,'J07': 61,'I07': 62,'I10': 63,'J10': 64,'D15': 65,'E15': 66,'G15': 67,'J15': 68,'L15': 69,'R15': 70,'T15': 71,'W15': 72,'I08': 73,'I03': 74,'J08': 75,'I01': 76,'I02': 77,'J01': 78,'K01': 79,'K02': 80,'L01': 81,'L02': 82,'M01': 83,'M02': 84,'N01': 85,'N02': 86,'O01': 87,'I09': 88,'D14': 89,'D13': 90,'K07': 91,'K08': 92,'N15': 93,'P15': 94,'I15': 95,'S15': 96,'U15': 97,'V15': 98,'S07': 99,'S08': 100,'L09': 101,'L08': 102,'Q02': 103,'Q01': 104}
df_full['location'] = df_full['location'].map(dict)
df_train=df_full.sample(frac=0.8,random_state=200)
df_valid=df_full.drop(df_train.index)

location_counts = BLE_RSSI.location.value_counts()
x1 = np.asarray(df_train[FEATURES])
y1 = np.asarray(df_train['location'])

x2 = np.asarray(df_valid[FEATURES])
y2 = np.asarray(df_valid['location'])

def formatFeatures(features):
    formattedFeatures = {}
    numColumns = features.shape[1]

    for i in range(0, numColumns):
        formattedFeatures["b"+str(3001+i)] = features[:, i]

    return formattedFeatures

trainingFeatures = formatFeatures(x1)
trainingCategories = y1

testFeatures = formatFeatures(x2)
testCategories = y2

# Train Input Function
def train_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((trainingFeatures, y1))
    dataset = dataset.repeat(1000).batch(TF_BATCH_SIZE)
    return dataset

# Test Input Function
def eval_input_fn():
    dataset = tf.data.Dataset.from_tensor_slices((testFeatures, y2))
    return dataset.repeat(1000).batch(TF_BATCH_SIZE)

# Provide list of GPUs should be used to train the model

distribution=tf.distribute.experimental.ParameterServerStrategy()
print('Number of devices: {}'.format(distribution.num_replicas_in_sync))

# Configuration of  training model

config = tf.estimator.RunConfig(train_distribute=distribution, model_dir=TF_MODEL_DIR, save_summary_steps=100, save_checkpoints_steps=100)

# Build 3 layer DNN classifier

model = tf.estimator.DNNClassifier(hidden_units = [13,65,110],
                 feature_columns = feature_columns,
                 model_dir = TF_MODEL_DIR,
                 n_classes=105, config=config
               )

export_final = tf.estimator.FinalExporter(TF_EXPORT_DIR, serving_input_receiver_fn=serving_input_receiver_fn)

train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, 
                                    max_steps=TF_TRAIN_STEPS)

eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn,
                                  steps=100,
                                  exporters=export_final,
                                  throttle_secs=1,
                                  start_delay_secs=1)

# Train and Evaluate the model

tf.estimator.train_and_evaluate(model, train_spec, eval_spec)

INFO:tensorflow:ParameterServerStrategy with compute_devices = ('/device:CPU:0',), variable_device = '/device:CPU:0'
Number of devices: 1
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Using config: {'_model_dir': 'blerssi/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.parameter_server_strategy.ParameterServerStrategyV1 object at 0x7f93ec2959b0>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec

({'accuracy': 0.24632813,
  'average_loss': 2.9123015,
  'loss': 372.7746,
  'global_step': 5000},
 [b'blerssi/export/blerssi/1595851517'])

## Define predict function

In [8]:
MODEL_EXPORT_PATH= os.path.join(TF_MODEL_DIR, "export", TF_EXPORT_DIR)

def predict(request):
    """ 
    Define custom predict function to be used by local prediction
    and explainer. Set anchor_tabular predict function so it always returns predicted class
    """
    # Get model exporter path
    for dir in os.listdir(MODEL_EXPORT_PATH):
        if re.match('[0-9]',dir):
            exported_path=os.path.join(MODEL_EXPORT_PATH,dir)
            break
    else:
        raise Exception("Model path not found")

    # Prepare model input data
    feature_cols=["b3001", "b3002","b3003","b3004","b3005","b3006","b3007","b3008","b3009","b3010","b3011","b3012","b3013"]
    input={'b3001': [], 'b3002': [], 'b3003': [], 'b3004': [], 'b3005': [], 'b3006': [], 'b3007': [], 'b3008': [], 'b3009': [], 'b3010': [], 'b3011': [], 'b3012': [], 'b3013': []}

    X=request
    if np.ndim(X) != 2:
        for i in range(len(X)):
            input[feature_cols[i]].append(X[i])
    else:
        for i in range(len(X)):
            for j in range(len(X[i])):
                input[feature_cols[j]].append(X[i][j])

    # Open a Session to predict
    with tf.Session() as sess:
        tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], exported_path)
        predictor= tf.contrib.predictor.from_saved_model(exported_path,signature_def_key='predict')
        output_dict= predictor(input)
    sess.close()
    output={}
    output["predictions"]={"probabilities":output_dict["probabilities"].tolist()}
    return np.asarray(output['predictions']["probabilities"])

## Initialize and fit
To initialize the explainer, we provide a predict function, a list with the feature names to make the anchors easy to understand.

In [9]:
feature_cols=["b3001", "b3002", "b3003", "b3004", "b3005", "b3006", "b3007", "b3008", "b3009", "b3010", "b3011", "b3012", "b3013"]
explainer = AnchorTabular(predict, feature_cols)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from blerssi/export/blerssi/1595851517/variables/variables
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Restoring parameters from blerssi/export/blerssi/1595851517/variables/variables


Discretize the ordinal features into quartiles. disc_perc is a list with percentiles used for binning

In [10]:
explainer.fit(x1, disc_perc=(25, 50, 75))

## Save Explainer file
Save explainer file with .dill extension. It will be used when creating the InferenceService

In [11]:
EXPLAINER_PATH="explainer"
if not os.path.exists(EXPLAINER_PATH):
    os.mkdir(EXPLAINER_PATH)
with open("%s/explainer.dill"%EXPLAINER_PATH, 'wb') as f:
    dill.dump(explainer,f)

## Create a gateway
Create a gateway called kubeflow-gateway in namespace anonymous.

In [12]:
gateway=f"""apiVersion: networking.istio.io/v1alpha3
kind: Gateway
metadata:
  name: kubeflow-gateway
  namespace: {namespace}
spec:
  selector:
    istio: ingressgateway
  servers:
  - hosts:
    - '*'
    port:
      name: http
      number: 80
      protocol: HTTP
"""
gateway_spec=yaml.safe_load(gateway)

In [13]:
custom_api.create_namespaced_custom_object(group="networking.istio.io", version="v1alpha3", namespace=namespace, plural="gateways", body=gateway_spec)

{'apiVersion': 'networking.istio.io/v1alpha3',
 'kind': 'Gateway',
 'metadata': {'creationTimestamp': '2020-07-27T12:05:50Z',
  'generation': 1,
  'name': 'kubeflow-gateway',
  'namespace': 'anonymous',
  'resourceVersion': '3341561',
  'selfLink': '/apis/networking.istio.io/v1alpha3/namespaces/anonymous/gateways/kubeflow-gateway',
  'uid': '6cf06d8c-f76e-472c-8a51-885e7ed6d701'},
 'spec': {'selector': {'istio': 'ingressgateway'},
  'servers': [{'hosts': ['*'],
    'port': {'name': 'http', 'number': 80, 'protocol': 'HTTP'}}]}}

## Adding a new inference server 
The list of available inference servers in Seldon Core is maintained in the **seldon-config** configmap, which lives in the same namespace as your Seldon Core operator. In particular, the **predictor_servers** key holds the JSON config for each inference server.

[Refer to for more information](https://docs.seldon.io/projects/seldon-core/en/v1.1.0/servers/custom.html)

In [14]:
api_client.patch_namespaced_config_map(name="seldon-config", namespace="kubeflow",pretty=True, body={"data":{"predictor_servers":'{"MLFLOW_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/mlflowserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/mlflowserver_rest"}},"SKLEARN_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/sklearnserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/sklearnserver_rest"}},"TENSORFLOW_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/tfserving-proxy_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/tfserving-proxy_rest"},"tensorflow":true,"tfImage":"tensorflow/serving:2.1.0"},"XGBOOST_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/xgboostserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/xgboostserver_rest"}}, "CUSTOM_INFERENCE_SERVER":{"rest":{"defaultImageVersion":"1.0","image":"samba07/blerssi-seldon"}}}'}})

{'api_version': 'v1',
 'binary_data': None,
 'data': {'credentials': '{"gcs":{"gcsCredentialFileName":"gcloud-application-credentials.json"},"s3":{"s3AccessKeyIDName":"awsAccessKeyID","s3SecretAccessKeyName":"awsSecretAccessKey"}}',
          'explainer': '{"image":"seldonio/alibiexplainer:1.2.1"}',
          'predictor_servers': '{"MLFLOW_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/mlflowserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/mlflowserver_rest"}},"SKLEARN_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/sklearnserver_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/sklearnserver_rest"}},"TENSORFLOW_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/tfserving-proxy_grpc"},"rest":{"defaultImageVersion":"1.2.1","image":"seldonio/tfserving-proxy_rest"},"tensorflow":true,"tfImage":"tensorflow/serving:2.1.0"},"XGBOOST_SERVER":{"grpc":{"defaultImageVersion":"1.2.1","image":"seldonio/xgboostserver_

## Seldon Serving Deployment
Create an **SeldonDeployment** with a blerssi model

In [15]:
pvcname = !(echo  $HOSTNAME | sed 's/.\{2\}$//')
pvc = "workspace-"+pvcname[0]
seldon_deploy=f"""apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: blerssi
  namespace: {namespace}
spec:
  name: blerssi
  predictors:
  - graph:
      children: []
      implementation: CUSTOM_INFERENCE_SERVER
      modelUri: pvc://{pvc}/{MODEL_EXPORT_PATH}
      name: blerssi
    explainer:
      containerSpec:
          image: seldonio/alibiexplainer:1.2.2-dev
          name: explainer
      type: AnchorTabular
      modelUri: pvc://{pvc}/{EXPLAINER_PATH}
    name: default
    replicas: 1
"""
seldon_deploy_spec=yaml.safe_load(seldon_deploy)

In [16]:
custom_api.create_namespaced_custom_object(group="machinelearning.seldon.io", version="v1alpha2", namespace=namespace, plural="seldondeployments", body=seldon_deploy_spec)

{'apiVersion': 'machinelearning.seldon.io/v1alpha2',
 'kind': 'SeldonDeployment',
 'metadata': {'creationTimestamp': '2020-07-27T12:05:57Z',
  'generation': 1,
  'name': 'blerssi',
  'namespace': 'anonymous',
  'resourceVersion': '3341586',
  'selfLink': '/apis/machinelearning.seldon.io/v1alpha2/namespaces/anonymous/seldondeployments/blerssi',
  'uid': 'fb1db218-fd85-4601-ae13-ec792f42387a'},
 'spec': {'name': 'blerssi',
  'predictors': [{'componentSpecs': [{'metadata': {'creationTimestamp': '2020-07-27T12:05:57Z'},
      'spec': {'containers': [{'image': 'samba07/blerssi-seldon:1.0',
         'name': 'blerssi',
         'ports': [{'containerPort': 6000,
           'name': 'metrics',
           'protocol': 'TCP'}],
         'resources': {},
         'volumeMounts': [{'mountPath': '/etc/podinfo',
           'name': 'seldon-podinfo'}]}]}}],
    'engineResources': {},
    'explainer': {'containerSpec': {'image': 'seldonio/alibiexplainer:1.2.2-dev',
      'name': 'explainer',
      'resour

## Wait for state to become available

In [17]:
status=False
while True:
    seldon_status=custom_api.get_namespaced_custom_object_status(group="machinelearning.seldon.io", version="v1alpha2", namespace=namespace, plural="seldondeployments", name=seldon_deploy_spec["metadata"]["name"])
    if seldon_status["status"]["state"] == "Available":
        status=True
        print("Status: %s"%seldon_status["status"]["state"])
    if status:
        break
    print("Status: %s"%seldon_status["status"]["state"])
    sleep(30)

Status: Creating
Status: Creating
Status: Available


## Run a Prediction

In [18]:
CLUSTER='ucs' #where your cluster running 'gcp' or 'ucs'

In [19]:
%%bash -s "$CLUSTER" --out NODE_IP
if [ $1 = "ucs" ]
then
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[0].address}')"
else
    echo "$(kubectl get node -o=jsonpath='{.items[0].status.addresses[1].address}')"
fi

In [20]:
%%bash --out INGRESS_PORT
INGRESS_GATEWAY="istio-ingressgateway"
echo "$(kubectl -n istio-system get service $INGRESS_GATEWAY -o jsonpath='{.spec.ports[1].nodePort}')"

### Data for prediction

In [21]:
df_full = pd.read_csv(os.path.join(path,'data/iBeacon_RSSI_Unlabeled_truncated.csv')) #Labeled dataset
  # Input Data Preprocessing 
df_full = df_full.drop(['date'],axis = 1)
df_full = df_full.drop(['location'],axis = 1)
df_full[FEATURES] = (df_full[FEATURES])/(-200)
input_data=df_full.to_numpy()[:1]
input_data

array([[1.   , 1.   , 0.4  , 1.   , 0.385, 0.28 , 0.405, 1.   , 1.   ,
        1.   , 1.   , 1.   , 1.   ]])

In [22]:
headers={"Content-Type": "application/json"}
def inference_predict(X):
    data={"data":{"ndarray":X.tolist()}}
    url = f"http://{NODE_IP.strip()}:{INGRESS_PORT.strip()}/seldon/{namespace}/%s/api/v1.0/predictions"%seldon_deploy_spec["metadata"]["name"]
    response=requests.post(url, data=json.dumps(data), headers=headers)
    probabilities=response.json()['data']['ndarray']
    for prob in probabilities:
        cls_id=np.argmax(prob)
        print("Probability: %s"%prob[cls_id])
        print("Class-id: %s"%cls_id)

def explain(X):
    if np.ndim(X)==2:
        data={"data":{"ndarray":X.tolist()}}
    else:
        data={"data":{"ndarray":[X.tolist()]}}
    url = f"http://{NODE_IP.strip()}:{INGRESS_PORT.strip()}/seldon/{namespace}/%s-explainer/default/api/v1.0/explain"%seldon_deploy_spec["metadata"]["name"]
    response=requests.post(url, data=json.dumps(data), headers=headers)
    print('Anchor: %s' % (' AND '.join(response.json()['names'])))
    print('Coverage: %.2f' % response.json()['coverage'])

In [23]:
inference_predict(input_data)

Probability: 0.6692667603492737
Class-id: 14


## Prediction of the model and explain

In [24]:
explain(input_data)

Anchor: b3009 <= 1.00 AND 0.40 < b3004 <= 1.00 AND 0.39 < b3002 <= 1.00 AND b3012 <= 1.00 AND b3011 <= 1.00 AND b3013 <= 1.00 AND b3006 <= 1.00 AND b3003 <= 1.00 AND b3010 <= 1.00 AND b3005 <= 1.00 AND b3001 <= 1.00 AND b3007 <= 1.00 AND b3008 <= 1.00
Coverage: 0.48


## Clean Up
### Delete a gateway

In [25]:
custom_api.delete_namespaced_custom_object(group="networking.istio.io", version="v1alpha3", namespace=namespace, plural="gateways", name=gateway_spec["metadata"]["name"],body=k8s_client.V1DeleteOptions())

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'kubeflow-gateway',
  'group': 'networking.istio.io',
  'kind': 'gateways',
  'uid': '6cf06d8c-f76e-472c-8a51-885e7ed6d701'}}

### Delete Seldon Serving Deployment

In [26]:
custom_api.delete_namespaced_custom_object(group="machinelearning.seldon.io", version="v1alpha2", namespace=namespace, plural="seldondeployments", name=seldon_deploy_spec["metadata"]["name"], body=k8s_client.V1DeleteOptions())

{'kind': 'Status',
 'apiVersion': 'v1',
 'metadata': {},
 'status': 'Success',
 'details': {'name': 'blerssi',
  'group': 'machinelearning.seldon.io',
  'kind': 'seldondeployments',
  'uid': 'fb1db218-fd85-4601-ae13-ec792f42387a'}}

### Delete model and explainer folders from notebook

In [27]:
!rm -rf $EXPLAINER_PATH
!rm -rf $TF_MODEL_DIR